# ML Ensamble introducing the Evaluator

In [2]:
import numpy as np

# ML Ensemble
from mlens.model_selection import Evaluator
from mlens.metrics import rmse

# Base Models
from sklearn.linear_model import Lasso
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor

# Preprocessing
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.pipeline import make_pipeline

# CV
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform, randint

In [3]:
np.random.seed(3675)

# Data
First, we need some data

In [4]:
# training data
X = np.random.random((1000, 10))

# noisy output, y = x0 * x1 + x2^2 + x3 - x4^(1/4) + e
y = X[:, 0] * X[:, 1] + X[:, 2] ** 2 + X[:, 3] - X[:, 4] ** (1 / 4)

# Change scales
X[:, 0] *= 10
X[:, 1] += 10
X[:, 2] *= 5
X[:, 3] *= 3
X[:, 4] /= 10

# Tackling model selection

Once a dataset has been at least moderately preprocessed, it's time to see what a learning algortihm can do. Often, we [don't know a priori](https://ti.arc.nasa.gov/m/profile/dhw/papers/78.pdf) which model will do well and sometimes, we even have different preprocessing pipelines that we would like to compare. This is when the `Evaluator` comes in handy. It allows the user to specify a set of
preprocessing cases to compare various estimators against. In the end, everything is summarized neatly in a DataFrame (or dict if you prefer) for a holistic overview. Neat, isn't it!

In [5]:
# A set of estimators to evaluate
ls = Lasso()
rf = RandomForestRegressor()
kn = KNeighborsRegressor()
sv = SVR()

# Some parameter distributions that might work well
ls_p = {'alpha': uniform(0.0005, 10)}
rf_p = {'max_depth': randint(2, 7), 'max_features': randint(3, 10),
        'min_samples_leaf': randint(2, 10)}
kn_p = {'n_neighbors': randint(2, 20)}
sv_p = {'C': uniform(0.01, 20)}

# Put it all in neat dictionaries. Note that the keys must match!
estimators = {'ls': ls, 'rf': rf, 'kn': kn, 'sv': sv}
parameters = {'ls': ls_p, 'rf': rf_p, 'kn': kn_p, 'sv': sv_p}

# A set of different preprocessing cases we want to try for each model
preprocessing = {'standard_scaler': [StandardScaler()], 'min_max': [MinMaxScaler()], 'None': []}

To initiate the `Evaluator`, simply call the class and pass the data to it along with
relevant grid-search information.

In [6]:
evals = Evaluator(X, y, preprocessing, rmse, cv=10, verbose=1,
                  n_jobs_estimators=-1, n_jobs_preprocessing=-1)

If preprocessing is heavy lifting we can pre-generate folds before evaluating any estimators by calling `preprocess`. (However, by evaluating
estimators their folds will automatically be stored, so you can safely run evaluation upon evaluation without having to worry about wading through heavy preprocessing.)

In [7]:
evals.preprocess()

[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:    0.0s finished


To evaluate a set of models, simply call the `evaluate` method along with the `estimators` and the `param_dicts`, the dictionary of parameter dictionaries.

In [8]:
evals.evaluate(estimators, parameters, n_iter=5)

Evaluating 4 models, with 5 parameter draws and 3 preprocessing options, over 10 CV folds, totalling fits 600.
[Parallel(n_jobs=-1)]: Done 232 tasks      | elapsed:    2.3s
[Parallel(n_jobs=-1)]: Done 600 out of 600 | elapsed:    5.6s finished
Evaluation done | 00:00:05



# Model evaluation made easy

With a single command we can see the performance of each estimator for each alterative preprocessing pipelines. Just call `summary_`

In [9]:
evals.summary_

,best_test_score_mean,best_test_score_std,train_score_mean,train_score_std,score_time,best_params,best_estimator,best_draw_idx
sv_min_max,-0.057709,0.00600658,-0.0536805,0.000789498,0.0355775,{'est__C': 19.8577695762},"Pipeline(steps=[('prep_1', MinMaxScaler(copy=T...",3
sv_standard_scaler,-0.101593,0.0072925,-0.0664077,0.000620663,0.0385854,{'est__C': 9.8360275689},"Pipeline(steps=[('prep_1', StandardScaler(copy...",2
sv_None,-0.110075,0.00755069,-0.0790476,0.000775603,0.176921,{'est__C': 19.8577695762},"Pipeline(steps=[('est', SVR(C=19.8577695762465...",3
rf_standard_scaler,-0.179369,0.00911402,-0.139811,0.00314456,0.0514906,"{'est__max_features': 9, 'est__min_samples_lea...","Pipeline(steps=[('prep_1', StandardScaler(copy...",1
rf_min_max,-0.181291,0.0112426,-0.139356,0.00332434,0.0503104,"{'est__max_features': 9, 'est__min_samples_lea...","Pipeline(steps=[('prep_1', MinMaxScaler(copy=T...",1
rf_None,-0.182032,0.0136991,-0.139598,0.00374242,0.0434263,"{'est__max_features': 9, 'est__min_samples_lea...","Pipeline(steps=[('est', RandomForestRegressor(...",1
kn_standard_scaler,-0.205662,0.0164844,-0.170119,0.0026078,0.00167966,{'est__n_neighbors': 6},"Pipeline(steps=[('prep_1', StandardScaler(copy...",4
kn_min_max,-0.208932,0.0163,-0.177899,0.00214726,0.000760865,{'est__n_neighbors': 7},"Pipeline(steps=[('prep_1', MinMaxScaler(copy=T...",3
kn_None,-0.226435,0.016165,-0.19393,0.00169686,0.000844598,{'est__n_neighbors': 7},"Pipeline(steps=[('est', KNeighborsRegressor(al...",3
ls_None,-0.498068,0.0258725,-0.497774,0.00289405,0.00531008,{'est__alpha': 3.9241479551},"Pipeline(steps=[('est', Lasso(alpha=3.92414795...",1
